# Widgets queries

In [1]:
import pandas as pd
import json
import requests
import numpy as np
import ipywidgets as widgets

## All crops

- WORLD PRICES FOR MAJOR COMMODITY CROPS
    * **Widget_id:** [9391e42c-f32b-40d9-a282-3083e617464f](https://api.resourcewatch.org/v1/widget/9391e42c-f32b-40d9-a282-3083e617464f)

In [7]:
payload = {'q': "select round((sum(value))::numeric,2) as val,avg(value) as val_avg, max_value, commodity, impactparameter, region, '2005 usd/Ton' as units from (SELECT  max(value) over (partition by impactparameter) max_value, * FROM combined01_prepared where scenario = 'SSP2-MIRO' and impactparameter = 'World price') r where year = 2010  and commodity IN  ('Maize','Rice','Soybean','Wheat') group by max_value, region, commodity, impactparameter order by impactparameter asc, commodity asc limit 5"}
r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df.head()

<Response [200]>


,commodity,impactparameter,max_value,region,units,val,val_avg
0,Maize,World price,2296.737549,-,2005 usd/Ton,148.66,148.657593
1,Rice,World price,2296.737549,-,2005 usd/Ton,357.08,357.076416
2,Soybean,World price,2296.737549,-,2005 usd/Ton,331.40,331.399841
3,Wheat,World price,2296.737549,-,2005 usd/Ton,219.44,219.442307


- AVERAGE GLOBAL CHANGE IN SEASONAL VARIABILITY FOR RAINFED CROPS THROUGH 2040
    * **Widget_id:** [abf5c0f9-4042-4ea6-9582-1b34209cb4ba](https://api.resourcewatch.org/v1/widget/abf5c0f9-4042-4ea6-9582-1b34209cb4ba)

`Sql:`
```sql
with f as (SELECT basinid, crop, irrigation, iso FROM crops where irrigation='rainfed'), 
     g as (SELECT basinid, sv3028tr, sv2028tr, sv4028tr, svbs00tr FROM aqueduct_projections_20150309  where  svbs00tr > -1), 
     a as (select avg(sv3028tr) as sv2030, avg(sv2028tr) as sv2020, avg(sv4028tr) as sv2040, avg(svbs00tr) as sv2010, crop from f,g where f.basinid=g.basinid group by crop), 
     b as (SELECT sv2010, sv2020, sv2030, sv2040, crop, 
           CASE when crop IN ('barley','maize','millet','rice','sorghum','wheat','other cereals') then 'Cereal' 
                when crop IN ('beans', 'cowpeas', 'chickpeas', 'groundnut', 'lentils', 'pigeonpeas', 'other pulses') then 'Pulses and legumes' 
                when crop IN ('cassava','potato','sweet potato','yams') then 'Roots And Tubers' 
                when crop IN ('banana','plantain') then 'Fruit and nuts' 
                when crop IN ('soybean') then 'Oilseed crops' END 
           as type FROM a t), 
     c as (select avg(sv2010) as sv2010,avg(sv2020) as sv2020,avg(sv2030) as sv2030,avg(sv2040) as sv2040, type from b group by type), 
   rrr as (SELECT type as cat, unnest(array[ '2010', '2020', '2030', '2040']) AS year, unnest(array[ sv2010, sv2020, sv2030, sv2040]) AS water_risk FROM c order by year, cat) 
           select year, cat as impactparameter, water_risk as value 
           FROM rrr WHERE cat IN ('Cereal', 'Pulses and legumes', 'Roots And Tubers', 'Fruit and nuts', 'Oilseed crops') ORDER BY year, 
                    CASE cat WHEN 'Cereal' THEN 1 
                             WHEN 'Pulses and legumes' THEN 2 
                             WHEN 'Roots And Tubers' THEN 3 
                             WHEN 'Fruit and nuts' THEN 4 
                             ELSE 5 END
```                                                                                    

In [13]:
payload = {'q': "with f as(SELECT basinid, crop, irrigation, iso FROM crops where irrigation='rainfed'), g as (SELECT basinid, sv3028tr, sv2028tr, sv4028tr, svbs00tr FROM aqueduct_projections_20150309  where  svbs00tr > -1), a as(select avg(sv3028tr) as sv2030, avg(sv2028tr) as sv2020, avg(sv4028tr) as sv2040, avg(svbs00tr) as sv2010, crop from f,g where f.basinid=g.basinid group by crop), b as (SELECT sv2010, sv2020, sv2030, sv2040, crop, CASE when crop IN ('barley','maize','millet','rice','sorghum','wheat','other cereals') then 'Cereal' when crop IN ('beans', 'cowpeas', 'chickpeas', 'groundnut', 'lentils', 'pigeonpeas', 'other pulses') then 'Pulses and legumes' when crop IN ('cassava','potato','sweet potato','yams') then 'Roots And Tubers' when crop IN ('banana','plantain') then 'Fruit and nuts' when crop IN ('soybean') then 'Oilseed crops' END as type FROM a t), c as (select avg(sv2010) as sv2010,avg(sv2020) as sv2020,avg(sv2030) as sv2030,avg(sv2040) as sv2040, type from b group by type), rrr as (SELECT type as cat, unnest(array[ '2010', '2020', '2030', '2040']) AS \"year\", unnest(array[ sv2010, sv2020, sv2030, sv2040]) AS \"water_risk\" FROM c order by year, cat) select year, cat as impactparameter, water_risk as value FROM rrr WHERE cat IN ('Cereal', 'Pulses and legumes', 'Roots And Tubers', 'Fruit and nuts', 'Oilseed crops') ORDER BY year, CASE cat WHEN 'Cereal' THEN 1 WHEN 'Pulses and legumes' THEN 2 WHEN 'Roots And Tubers' THEN 3 WHEN 'Fruit and nuts' THEN 4 ELSE 5 END"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df[df['year'] == 2010]

<Response [200]>


,impactparameter,value,year
0,Cereal,0.911941,2010
1,Pulses and legumes,1.084655,2010
2,Roots And Tubers,0.911385,2010
3,Fruit and nuts,0.902324,2010
4,Oilseed crops,0.797071,2010


`New sql:`
```sql
with f as (SELECT basinid, crop, irrigation, iso FROM crops_projected where irrigation='rainfed'), 
     g as (SELECT basinid, sv3028tr, sv2028tr, sv4028tr, svbs00tr FROM aqueduct_projections_20150309  where  svbs00tr > -1), 
     a as (select avg(sv3028tr) as sv2030, avg(sv2028tr) as sv2020, avg(sv4028tr) as sv2040, avg(svbs00tr) as sv2010, crop from f,g where f.basinid=g.basinid group by crop), 
     b as (SELECT sv2010, sv2020, sv2030, sv2040, crop, 
           CASE when crop IN ('barley','maize','small millet', 'pearl millet','rice','sorghum','wheat','other cereals') then 'Cereals' 
                when crop IN ('bean', 'cowpea', 'chickpea', 'pigeonpea', 'lentil', 'other pulses') then 'Pulses and legumes' 
                when crop IN ('cassava','potato','sweet potato','yams', 'other roots') then 'Roots and tubers' 
                when crop IN ('banana','plantain', 'tropical fruit', 'temperate fruit') then 'Fruit and nuts' 
                when crop IN ('soybean', 'groundnut', 'coconut', 'oilpalm', 'sunflower', 'rapeseed', 'sesameseed', 'other oil crops') then 'Oilseed crops'  
                when crop IN ('sugarcane', 'sugarbeet') then 'Sugar crops' 
                when crop IN ('cotton', 'other fibre crops') then 'Fibres'  
                when crop IN ('arabica coffee', 'robusta coffee', 'cocoa', 'tea', 'tobacco') then 'Stimulants'  
                when crop IN ('vegetables') then 'Vegetables' 
                when crop IN ('rest of crops') then 'Other crops' END 
           as type FROM a t), 
     c as (select avg(sv2010) as sv2010,avg(sv2020) as sv2020,avg(sv2030) as sv2030,avg(sv2040) as sv2040, type from b group by type), 
   rrr as (SELECT type as cat, unnest(array[ '2010', '2020', '2030', '2040']) AS year, unnest(array[ sv2010, sv2020, sv2030, sv2040]) AS water_risk FROM c order by year, cat) 
           select year, cat as impactparameter, water_risk as value 
           FROM rrr WHERE cat IN ('Cereals', 'Pulses and legumes', 'Roots and tubers', 'Fruit and nuts', 'Oilseed crops', 'Sugar crops', 'Fibres', 'Stimulants', 'Vegetables', 'Other crops') ORDER BY year, 
                    CASE cat WHEN 'Cereals' THEN 1 
                             WHEN 'Pulses and legumes' THEN 2 
                             WHEN 'Roots and tubers' THEN 3 
                             WHEN 'Fruit and nuts' THEN 4 
                             WHEN 'Oilseed crops' THEN 5
                             WHEN 'Sugar crops' THEN 6 
                             WHEN 'Fibres' THEN 7
                             WHEN 'Stimulants' THEN 8
                             WHEN 'Vegetables' THEN 9
                             WHEN 'Other crops' THEN 10
                             ELSE 11 END
```

In [11]:
payload = {'q': "with f as (SELECT basinid, crop, irrigation, iso FROM crops_projected where irrigation='rainfed'), g as (SELECT basinid, sv3028tr, sv2028tr, sv4028tr, svbs00tr FROM aqueduct_projections_20150309  where  svbs00tr > -1), a as (select avg(sv3028tr) as sv2030, avg(sv2028tr) as sv2020, avg(sv4028tr) as sv2040, avg(svbs00tr) as sv2010, crop from f,g where f.basinid=g.basinid group by crop), b as (SELECT sv2010, sv2020, sv2030, sv2040, crop, CASE when crop IN ('barley','maize','small millet', 'pearl millet','rice','sorghum','wheat','other cereals') then 'Cereals' when crop IN ('bean', 'cowpea', 'chickpea', 'pigeonpea', 'other pulses') then 'Pulses and legumes' when crop IN ('cassava','potato','sweet potato','yams', 'other roots') then 'Roots and tubers' when crop IN ('banana','plantain', 'tropical fruit', 'temperate fruit') then 'Fruit and nuts' when crop IN ('soybean', 'groundnut', 'coconut', 'oilpalm', 'sunflower', 'rapeseed', 'sesameseed', 'other oil crops') then 'Oilseed crops'  when crop IN ('sugarcane', 'sugarbeet') then 'Sugar crops' when crop IN ('cotton', 'other fibre crops') then 'Fibres'  when crop IN ('arabica coffee', 'robusta coffee', 'cocoa', 'tea', 'tobacco') then 'Stimulants'  when crop IN ('vegetables') then 'Vegetables' when crop IN ('rest of crops') then 'Other crops' END as type FROM a t), c as (select avg(sv2010) as sv2010,avg(sv2020) as sv2020,avg(sv2030) as sv2030,avg(sv2040) as sv2040, type from b group by type), rrr as (SELECT type as cat, unnest(array[ '2010', '2020', '2030', '2040']) AS \"year\", unnest(array[ sv2010, sv2020, sv2030, sv2040]) AS \"water_risk\" FROM c order by year, cat) select year, cat as impactparameter, water_risk as value FROM rrr WHERE cat IN ('Cereals', 'Pulses and legumes', 'Roots and tubers', 'Fruit and nuts', 'Oilseed crops', 'Sugar crops', 'Fibres', 'Stimulants', 'Vegetables', 'Other crops') ORDER BY year, CASE cat WHEN 'Cereals' THEN 1  WHEN 'Pulses and legumes' THEN 2  WHEN 'Roots and tubers' THEN 3  WHEN 'Fruit and nuts' THEN 4  WHEN 'Oilseed crops' THEN 5 WHEN 'Sugar crops' THEN 6  WHEN 'Fibres' THEN 7 WHEN 'Stimulants' THEN 8 WHEN 'Vegetables' THEN 9 WHEN 'Other crops' THEN 10 ELSE 11 END"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df[df['year'] == 2010]

<Response [200]>


,impactparameter,value,year
0,Cereals,0.920478,2010
1,Pulses and legumes,1.071176,2010
2,Roots and tubers,0.915719,2010
3,Fruit and nuts,0.870287,2010
4,Oilseed crops,0.887110,2010
5,Sugar crops,0.800522,2010
6,Fibres,0.967228,2010
7,Stimulants,0.842587,2010
8,Vegetables,0.846058,2010
9,Other crops,0.928133,2010


- PERCENTAGE OF CROP AREA BY LEVEL OF WATER STRESS AND VOLUME OF DEMAND
    * **Widget_id:** [7b29a2a5-515a-4e22-846f-03ad3d2c0a13](https://api.resourcewatch.org/v1/widget/7b29a2a5-515a-4e22-846f-03ad3d2c0a13)

In [15]:
payload = {'q': "with t as (select sum(area) as area, ws4028tl as risk, crop from crops_stats where irrigation='irrigated' group by ws4028tl, crop order by risk asc), r as (SELECT sum(value) as value, commodity FROM combined01_prepared WHERE  commodity in ('Maize','Rice','Soybean','Wheat') and scenario = 'SSP2-MIRO' and region='World' and year=2020 and impactparameter='Food Demand'  group by commodity ), d as ( select value, commodity from r ) select crop, risk,value, (area*100/sum(area) over(partition by crop)) area_perc from d inner join t on lower(commodity)=crop order by risk asc, crop asc"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df.head()

<Response [200]>


,area_perc,crop,risk,value
0,0.084158,maize,1.No Data,132734.593750
1,0.088294,rice,1.No Data,372386.968750
2,0.306822,soybean,1.No Data,18094.544922
3,0.083799,wheat,1.No Data,502489.718750
4,24.664719,maize,2.Low,132734.593750


- AVERAGE GLOBAL CHANGE IN WATER STRESS FOR IRRIGATED CROPS THROUGH 2040
    * **Widget_id:** [4f322f4a-dde6-4f63-b7c9-10b0ba0c1402](https://api.resourcewatch.org/v1/widget/4f322f4a-dde6-4f63-b7c9-10b0ba0c1402)

`Sql:`
```sql
with alpha as (SELECT crop, year, value FROM water_risk_rankings_v3 where indicator='water_stress' and irrigation='irrigated' and value is not null order by year asc), 
     betha as (select value, year, 
               CASE when crop IN ('barley','maize','millet','rice','sorghum','wheat','millet_small','millet_pearl', 'cereals_other') then 'Cereal' 
                    when crop IN ('beans', 'cowpeas', 'chickpeas', 'groundnut', 'lentils', 'pigeonpeas', 'pulses_other') then 'Pulses and legumes' 
                    when crop IN ('cassava','potato','sweet potato','yams') then 'Roots And Tubers' 
                    when crop IN ('banana','plantain') then 'Fruit and nuts' 
                    when crop IN ('soybean') then 'Oilseed crops' END 
               as grupo from alpha order by year asc, grupo) 
               
select avg(value) as water_risk, year, grupo as cat 
from betha 
where grupo is not null 
group by grupo, year 
```

In [18]:
payload = {'q': "with alpha as (SELECT crop, year, value FROM water_risk_rankings_v3 where indicator='water_stress' and irrigation='irrigated' and value is not null order by year asc), betha as (select value, year, CASE when crop IN ('barley','maize','millet','rice','sorghum','wheat','millet_small','millet_pearl', 'cereals_other') then 'Cereal' when crop IN ('beans', 'cowpeas', 'chickpeas', 'groundnut', 'lentils', 'pigeonpeas', 'pulses_other') then 'Pulses and legumes' when crop IN ('cassava','potato','sweet potato','yams') then 'Roots And Tubers' when crop IN ('banana','plantain') then 'Fruit and nuts' when crop IN ('soybean') then 'Oilseed crops' END as grupo from alpha order by year asc, grupo) select avg(value) as water_risk, year, grupo as cat from betha where grupo is not null group by grupo, year order by year, grupo"}
r = requests.get('https://wri-rw.carto.com/api/v2/sql', params=payload)
print(r)
df = pd.read_json(json.dumps(r.json()['rows']), orient='records')
df

<Response [200]>


,cat,water_risk,year
0,Cereal,1.321041,2014
1,Fruit and nuts,1.060130,2014
2,Oilseed crops,1.004320,2014
3,Pulses and legumes,1.714746,2014
4,Roots And Tubers,1.424695,2014
5,Cereal,2.321000,2020
6,Fruit and nuts,1.689004,2020
7,Oilseed crops,1.702955,2020
8,Pulses and legumes,2.649582,2020
9,Roots And Tubers,2.107380,2020


`Sql: 2010`
```sql
with a as (SELECT coalesce(bws_score, -9999) as  value, r.crop, r.area
           FROM water_risk_indicators_annual_all s
           LEFT JOIN crops_baseline r on s.aq30_id=r.aq30_id
           WHERE s.pfaf_id != -9999 and s.gid_1 != '-9999' and  bws_score != -9999 and irrigation = 'irrigated'),
     b as (select value, area, crop,  
           CASE when crop IN ('barley','maize','small millet', 'pearl millet','rice','sorghum','wheat','other cereals') then 'Cereals' 
                when crop IN ('bean', 'cowpea', 'chickpea', 'pigeonpea', 'lentil', 'other pulses') then 'Pulses and legumes' 
                when crop IN ('cassava','potato','sweet potato','yams', 'other roots') then 'Roots and tubers' 
                when crop IN ('banana','plantain', 'tropical fruit', 'temperate fruit') then 'Fruit and nuts' 
                when crop IN ('soybean', 'groundnut', 'coconut', 'oilpalm', 'sunflower', 'rapeseed', 'sesameseed', 'other oil crops') then 'Oilseed crops'  
                when crop IN ('sugarcane', 'sugarbeet') then 'Sugar crops' 
                when crop IN ('cotton', 'other fibre crops') then 'Fibres'  
                when crop IN ('arabica coffee', 'robusta coffee', 'cocoa', 'tea', 'tobacco') then 'Stimulants'  
                when crop IN ('vegetables') then 'Vegetables' 
                when crop IN ('rest of crops') then 'Other crops' END 
           as grupo FROM a)
 
SELECT avg(value*area)/avg(area) as water_risk, grupo as cat
FROM b 
WHERE grupo is not null 
GROUP BY grupo
```

`Sql: 2010`
```sql
with a as (SELECT coalesce(bws_raw, -9999) as  value, r.crop, r.area
           FROM water_risk_indicators_annual_all s
           LEFT JOIN crops_baseline r on s.aq30_id=r.aq30_id
           WHERE s.pfaf_id != -9999 and s.gid_1 != '-9999' and  bws_raw != -9999 and irrigation = 'irrigated'),
     b as (select value, area, crop,  
           CASE when crop IN ('barley','maize','small millet', 'pearl millet','rice','sorghum','wheat','other cereals') then 'Cereals' 
                when crop IN ('bean', 'cowpea', 'chickpea', 'pigeonpea', 'lentil', 'other pulses') then 'Pulses and legumes' 
                when crop IN ('cassava','potato','sweet potato','yams', 'other roots') then 'Roots and tubers' 
                when crop IN ('banana','plantain', 'tropical fruit', 'temperate fruit') then 'Fruit and nuts' 
                when crop IN ('soybean', 'groundnut', 'coconut', 'oilpalm', 'sunflower', 'rapeseed', 'sesameseed', 'other oil crops') then 'Oilseed crops'  
                when crop IN ('sugarcane', 'sugarbeet') then 'Sugar crops' 
                when crop IN ('cotton', 'other fibre crops') then 'Fibres'  
                when crop IN ('arabica coffee', 'robusta coffee', 'cocoa', 'tea', 'tobacco') then 'Stimulants'  
                when crop IN ('vegetables') then 'Vegetables' 
                when crop IN ('rest of crops') then 'Other crops' END 
           as grupo FROM a)
 
SELECT avg(value*area)/avg(area) as water_risk, grupo as cat
FROM b 
WHERE grupo is not null 
GROUP BY grupo
```

`Sql 2030:`
```sql
with a as (SELECT value , r.crop, r.area
           FROM water_risk_indicators_projections s
           LEFT JOIN crops_projected r on s.basinid=r.basinid
           WHERE type = 'future_value' and indicator = 'water_stress' and year = 2030 and scenario = 'business_as_usual'),
     b as (select value, area, crop,  
           CASE when crop IN ('barley','maize','small millet', 'pearl millet','rice','sorghum','wheat','other cereals') then 'Cereals' 
                when crop IN ('bean', 'cowpea', 'chickpea', 'pigeonpea', 'lentil', 'other pulses') then 'Pulses and legumes' 
                when crop IN ('cassava','potato','sweet potato','yams', 'other roots') then 'Roots and tubers' 
                when crop IN ('banana','plantain', 'tropical fruit', 'temperate fruit') then 'Fruit and nuts' 
                when crop IN ('soybean', 'groundnut', 'coconut', 'oilpalm', 'sunflower', 'rapeseed', 'sesameseed', 'other oil crops') then 'Oilseed crops'  
                when crop IN ('sugarcane', 'sugarbeet') then 'Sugar crops' 
                when crop IN ('cotton', 'other fibre crops') then 'Fibres'  
                when crop IN ('arabica coffee', 'robusta coffee', 'cocoa', 'tea', 'tobacco') then 'Stimulants'  
                when crop IN ('vegetables') then 'Vegetables' 
                when crop IN ('rest of crops') then 'Other crops' END 
           as grupo FROM a)
 
SELECT avg(value) as water_risk, grupo as cat
FROM b 
WHERE grupo is not null 
GROUP BY grupo
 
```

- PERCENTAGE OF GLOBAL CROP AREA THAT IS IRRIGATED VS. RAINFED
    * **Widget_id:** [25facdf3-acf6-4f0e-9a0a-5ec94216407f](https://api.resourcewatch.org/v1/widget/25facdf3-acf6-4f0e-9a0a-5ec94216407f)
- PERCENTAGE OF GLOBAL CROP PRODUCTION AREA BY WATER RISK
    * **Widget_id:** [7b37bd6e-a465-4c4d-98b3-1a110a66d27e](https://api.resourcewatch.org/v1/widget/7b37bd6e-a465-4c4d-98b3-1a110a66d27e)

## One crop

- There are 22M hectares of irrigated {{crop}} growing under high water stress conditions.
    * **Widget_id:** [f6f18cd2-ce95-4436-af73-32378ebaa551](https://api.resourcewatch.org/v1/widget/f6f18cd2-ce95-4436-af73-32378ebaa551)
- PERCENTAGE OF GLOBAL {{crop}} AREA THAT IS IRRIGATED VS. RAINFED
    * **Widget_id:** [25facdf3-acf6-4f0e-9a0a-5ec94216407f](https://api.resourcewatch.org/v1/widget/25facdf3-acf6-4f0e-9a0a-5ec94216407f)
- TOP 5 NET IMPORTERS AND NET EXPORTERS OF {{crop}}
    * **Widget_id:** [4a3582e3-65d8-4331-b397-f42ed5d190d0](https://api.resourcewatch.org/v1/widget/4a3582e3-65d8-4331-b397-f42ed5d190d0)
- TOP 10 COUNTRIES WITH THE HIGHEST {{crop}} WATER STRESS SCORES
    * **Widget_id:** [413a36b5-a7b4-4bb4-9491-98b5175b4e28](https://api.resourcewatch.org/v1/widget/413a36b5-a7b4-4bb4-9491-98b5175b4e28)
- TOP 10 COUNTRIES WITH THE HIGHEST {{crop}} DROUGHT SEVERITY RISK SCORES
    * **Widget_id:** [51e8d799-52f9-48cc-b91c-bd21d7ae57f6](https://api.resourcewatch.org/v1/widget/51e8d799-52f9-48cc-b91c-bd21d7ae57f6)
- GLOBAL {{crop}} YIELD, AREA, PRODUCTION, PRICE, AND DEMAND CHANGE OVER TIME
    * **Widget_id:** [e0bdc2fc-e1a9-431e-ad91-5f2c9023f03f](https://api.resourcewatch.org/v1/widget/e0bdc2fc-e1a9-431e-ad91-5f2c9023f03f)
- PERCENTAGE OF GLOBAL {{crop}} PRODUCTION AREA BY WATER RISK
    * **Widget_id:** [7b37bd6e-a465-4c4d-98b3-1a110a66d27e](https://api.resourcewatch.org/v1/widget/7b37bd6e-a465-4c4d-98b3-1a110a66d27e)